# Object tracking in Python
**Discuss the outline of the session and how this notebook will work**
- **Talk about how the general structure for an example tracking workflow has been prepared, but the various functions have gaps that will need to be filled in**
- **Mention that the Notebook is broken down into sections.  Each one starts with a bit of background information and a list of what we will achieve at the end of it**

## Getting started
### Background
Before we start, we need to make sure this Jupyter Notebook has access to all the functions we're going to use later on.  Many of these will already be included with your Python build; however, a few may be missing.  To deal with this, we first run Pip to download these libraries.  After this, we can do a standard Python import.

**Discuss what each import is used for**

### Aim
- Install missing Python libraries
- Import any functions we will use later on

In [ ]:
# Running pip against all the libraries we will need.  If these aren't already present, they will be downloaded.
!pip install --user matplotlib
!pip install --user pandas
!pip install --user Pillow
!pip install --user scikit-image
!pip install --user opencv-python

In [ ]:
# Importing the libraries into this Notebook
import math
import sys
import time
import util

import numpy as np

from scipy.optimize import linear_sum_assignment

## Ex 1 - Loading coordinates and visualising
### Background
- The aim of the game this afternoon is to learn about tracking, so we don't want to spend half our time detecting objects to be tracked.  As such, you'll find a pre-prepared CSV file with object coordinates at **../data/ExampleCoordinates.csv**
- **Discuss the contents of the csv such as columns**
- The image these coordinates were extracted from is at **../data/ExampleTimeseries.tif**
- Both the example image and associated coordinates were taken from **Where?**
- Similarly, we could sink a couple of hours into creating a script to draw our object coordinates and tracks.  For now, I've created a separate script which will do this for us - this is in the "util.py" file we just imported.

### Aim
- Load object coordinates from **../data/ExampleCoordinate.csv** into a Pandas dataframe
- Load the timeseries images corresponding to the coordinates into a 3D Numpy array
- Visualise the coordinates as an overlay on the timeseries images

In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

In [ ]:
%matplotlib notebook

# Loading image stack
path = "../data/ExampleTimeseries.tif"
images = util.load_images(path);

# Loading coordinates
path = "../data/ObjectCoordinatesNoHeader.csv"
coords = util.load_coordinates(path);

# Adding track renders
util.show_overlay(images,coords,False)

## Ex 2 - Extracting coordinates for a specific timepoint
We have the full list of coordinates, but for tracking we're often going to want to access just those from a specific frame.  The first function we'll create will take the full list of coordinates and return a **list** of rows corresponding to only those points in the specified frame .

In [ ]:
### DESCRIBE THIS METHOD ###
def get_current_coords(coords, frame):
    # Identifying rows of "coords" with current frame number
    rows = coords.index[coords.FRAME == frame]

    return rows

In [ ]:
### IMPLEMENT THIS WITH OUR COORDINATES ### 
# Loading coordinates
path = "../data/ObjectCoordinatesNoHeader.csv"
coords = util.load_coordinates(path);

# Testing on coordinates from frame 0
frame = 0
rows_0 = get_current_coords(coords,frame)
print("Row indices for points in frame 0:")
print(rows_0)

# Testing on coordinates from frame 1
frame = 1
rows_1 = get_current_coords(coords,frame)
print("Row indices for points in frame 1")
print(rows_1)

## Ex 3 - Assign tracks IDs to the coordinates in the first frame
Eventually, all points will be assigned track IDs.  As we work through all the frames, points will be linked to those in a previous frame and inherit their track IDs.  This way, track ID gets propagated through the timeseries.  To kick things off, we need to assign all points in the first frame unique IDs.  The function we create here will identify any points in a specific frame (for now, frame 0) that don't have assigned track IDs and assign them the smallest, currently unused track ID.  By keeping the function general like this we can reuse it later on when find any points in any frame that didn't get linked back to an existing track.

## Ex 4 - Getting all available tracks
When assigning track links for each frame, we need to know what tracks are available to link to.  Here, we want to identify the rows corresponding to the most recent instance of each track.  We may also choose to only allow links to tracks identified within a specific number of frames (i.e. we may not want to allow a point in frame 42 to link back to a track last seen in frame 9).

## Ex 5 - Calculating cost for linking two points
In a couple of steps we will end up with two lists: one for points in the current frame; the other for the most recent point in all available tracks.  Links between the points in the two lists will be assigned based on the cost associated with making that link.  Here, we want to create a function that will calculate the cost associated with two given points.  To start with, we will just calculate the cost as the distance between the two points, but we could also add costs associated with other metrics.  For example, we may want to penalise links which would see the size or intensity of the object change too much.

**Add a figure demonstrating mislinking objects of different sizes, then how a size-change term could favour correct assignment**

## Ex 6 - Calculate cost matrix
Now we can calculate the cost for a single pair of points we need to do it for all point pairs.  We will create a function which takes the points in the current frame and the available track points, then generates a 2D cost matrix.  The cost matrix will have a **row** for each point in the current frame and a **column** for each available track.  The value of each element will therefore be the cost of linking the point and track corresponding to that row and column.  We also want to limit the distance that links can be made over; therefore, any point-track pairs separations that are greater than a specific distance will be set to **infinity**.  This doesn't prevent them being linked (if there are no better options, the assignment algorithm will still suggest that as a link), but it makes it less likely.

## Ex 7 - Calculate assignments
The job of calculating the assignments is done using the Munkres (aka. Kuhn-Munkres or Hungarian) algorithm.  Rather than create our own implementation of this, we will use SciPy's linear_sum_assignment function.  This algorithm simply takes the cost matrix we just created and outputs a 1D **list** of the assignments.  Each element of this list corresponds to a **row** (**i.e. current point**) in the cost matrix, while the value of that element corresponds to the assigned **column** (**i.e. track**).  We use this list of assignments to assign the current points the correct track ID.  As mentioned previously, the Munkres algorithm will still assign links that we set to **infinity**, so before we copy over any track IDs we want to double check the cost is not **infinity**.

## Ex 8 - Assigning track IDs to unlinked points
Any unlinked points in the current frame will still have a track ID of 0.  We want to find these and assign them the next available track ID.  To do this, we can reuse the function we created earlier to assign track IDs to the points in the first frame.

## Ex 9 - Putting it all together
We now have all the components necessary to construct a full tracking workflow.  For this, we put most of the components in a single for-loop, which will iterate over each frame.  At the end of this we will re-render the overlay to see if our tracking has worked correctly.

## Ex 10 - Quantifying our tracks
With a fully tracked set of coordinates we probably want to make some measurements.  Here we will look at a few common metrics.